# CIAN parser by Elsakova Anna

Loading libraries

In [1]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import math
import time

In [2]:
def html_stripper(text):
    return re.sub('<[^<]+?>', '', str(text))

## Find links to each okrug's flats

In [3]:
#all_flats = 'http://www.cian.ru/cat.php?deal_type=sale&engine_version=2&offer_type=flat&p={}&region=1&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room7=1&room9=1'
AO = [None]*11
#CAO
AO[0] = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=13&district%5B1%5D=14&district%5B2%5D=15&district%5B3%5D=16&district%5B4%5D=17&district%5B5%5D=18&district%5B6%5D=19&district%5B7%5D=20&district%5B8%5D=21&district%5B9%5D=22&engine_version=2&offer_type=flat&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room7=1&room9=1'
#SAO
AO[1] = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=23&district%5B10%5D=33&district%5B11%5D=34&district%5B12%5D=35&district%5B13%5D=36&district%5B14%5D=37&district%5B15%5D=38&district%5B1%5D=24&district%5B2%5D=25&district%5B3%5D=26&district%5B4%5D=27&district%5B5%5D=28&district%5B6%5D=29&district%5B7%5D=30&district%5B8%5D=31&district%5B9%5D=32&engine_version=2&offer_type=flat&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room7=1&room9=1'
#SVAO
AO[2] = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=39&district%5B10%5D=49&district%5B11%5D=50&district%5B12%5D=51&district%5B13%5D=52&district%5B14%5D=53&district%5B15%5D=54&district%5B16%5D=55&district%5B1%5D=40&district%5B2%5D=41&district%5B3%5D=42&district%5B4%5D=43&district%5B5%5D=44&district%5B6%5D=45&district%5B7%5D=46&district%5B8%5D=47&district%5B9%5D=48&engine_version=2&offer_type=flat&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room7=1&room9=1'
#VAO
AO[3] = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=56&district%5B10%5D=66&district%5B11%5D=67&district%5B12%5D=68&district%5B13%5D=69&district%5B14%5D=70&district%5B15%5D=71&district%5B1%5D=57&district%5B2%5D=58&district%5B3%5D=59&district%5B4%5D=60&district%5B5%5D=61&district%5B6%5D=62&district%5B7%5D=63&district%5B8%5D=64&district%5B9%5D=65&engine_version=2&offer_type=flat&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room7=1&room9=1'
#UVAO
AO[4] = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=72&district%5B10%5D=82&district%5B11%5D=83&district%5B1%5D=73&district%5B2%5D=74&district%5B3%5D=75&district%5B4%5D=76&district%5B5%5D=77&district%5B6%5D=78&district%5B7%5D=79&district%5B8%5D=80&district%5B9%5D=81&engine_version=2&offer_type=flat&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room7=1&room9=1'
#UAO
AO[5] = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=84&district%5B10%5D=94&district%5B11%5D=95&district%5B12%5D=96&district%5B13%5D=97&district%5B14%5D=98&district%5B15%5D=99&district%5B1%5D=85&district%5B2%5D=86&district%5B3%5D=87&district%5B4%5D=88&district%5B5%5D=89&district%5B6%5D=90&district%5B7%5D=91&district%5B8%5D=92&district%5B9%5D=93&engine_version=2&offer_type=flat&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room7=1&room9=1'
#UZAO
AO[6] = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=100&district%5B10%5D=110&district%5B11%5D=111&district%5B1%5D=101&district%5B2%5D=102&district%5B3%5D=103&district%5B4%5D=104&district%5B5%5D=105&district%5B6%5D=106&district%5B7%5D=107&district%5B8%5D=108&district%5B9%5D=109&engine_version=2&offer_type=flat&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room7=1&room9=1'
#ZAO
AO[7] = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=112&district%5B10%5D=122&district%5B11%5D=123&district%5B12%5D=124&district%5B13%5D=348&district%5B14%5D=349&district%5B15%5D=350&district%5B1%5D=113&district%5B2%5D=114&district%5B3%5D=115&district%5B4%5D=116&district%5B5%5D=117&district%5B6%5D=118&district%5B7%5D=119&district%5B8%5D=120&district%5B9%5D=121&engine_version=2&offer_type=flat&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room7=1&room9=1'
#SZAO
AO[8] = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=125&district%5B1%5D=126&district%5B2%5D=127&district%5B3%5D=128&district%5B4%5D=129&district%5B5%5D=130&district%5B6%5D=131&district%5B7%5D=132&engine_version=2&offer_type=flat&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room7=1&room9=1'
#NAO
AO[9] = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=327&district%5B10%5D=337&district%5B1%5D=328&district%5B2%5D=329&district%5B3%5D=330&district%5B4%5D=331&district%5B5%5D=332&district%5B6%5D=333&district%5B7%5D=334&district%5B8%5D=335&district%5B9%5D=336&engine_version=2&offer_type=flat&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room7=1&room9=1'
#TAO
AO[10] = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=338&district%5B1%5D=339&district%5B2%5D=340&district%5B3%5D=341&district%5B4%5D=342&district%5B5%5D=343&district%5B6%5D=344&district%5B7%5D=345&district%5B8%5D=346&district%5B9%5D=347&engine_version=2&offer_type=flat&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room7=1&room9=1'
#As ZelAO have less pages than other okrugs, then we will parse it in another way
ZelAO_url = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=152&district%5B1%5D=153&district%5B2%5D=154&district%5B3%5D=355&district%5B4%5D=356&district%5B5%5D=357&district%5B6%5D=358&engine_version=2&offer_type=flat&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room7=1&room9=1'
ZelAO_numofpages = 1

## Loading flat links 

In [4]:
def load_flats_from_one_okrug(okrug_url, okrug_number_of_pages, links):
    for page in range(1,okrug_number_of_pages+1):
        page_url =  okrug_url.format(page)

        search_page = requests.get(page_url)
        search_page = search_page.content
        search_page = BeautifulSoup(search_page, 'lxml')

        flat_urls = search_page.findAll('div', attrs = {'ng-class':"{'serp-item_removed': offer.remove.state, 'serp-item_popup-opened': isPopupOpen}"})
        flat_urls = re.split('http://www.cian.ru/sale/flat/|/" ng-class="', str(flat_urls))

        for link in flat_urls:
            if link.isdigit():
                links.append(link)
    return links

In [5]:
links = []
for i in range(0,len(AO)):
    load_flats_from_one_okrug(AO[i],30,links);
    print('Okrug', i, 'is loaded')
load_flats_from_one_okrug(ZelAO_url,ZelAO_numofpages,links);

Okrug 0 is loaded
Okrug 1 is loaded
Okrug 2 is loaded
Okrug 3 is loaded
Okrug 4 is loaded
Okrug 5 is loaded
Okrug 6 is loaded
Okrug 7 is loaded
Okrug 8 is loaded
Okrug 9 is loaded
Okrug 10 is loaded


In [6]:
len(links)

9244

## Parsing begins!

### Flat URL number and ordinal number

In [7]:
#for each flat remember it's url number and ordinal number
flats_info = [{'Flat URL': links[i],'N': i} for i in range(len(links))]
flats_info[0:3]

[{'Flat URL': '150919303', 'N': 0},
 {'Flat URL': '150425454', 'N': 1},
 {'Flat URL': '149914299', 'N': 2}]

There are some functions that we would need for parsing information.
For parsing information I supposed, that everywhere where "–" is written it means that that data is missing, not the zero mening of variable. It is absolutely true for info about space or metro I think (somewhere an information about kitchen or live space is missing). And for balcony and telephone I will think that if there written "no" or zero it means absence of it, and "–" means missing data.

In [8]:
#this function distinguish a number (or no information) in a string
def parse_quantitative_info( string ):
    #print(string)
    num = re.search(r'([–0-9,]+)', string)
    if num: #if there were appropriate information - not strings
        num = num.groups()[0].replace(',','.') #find the number in the string
        if num == '–': #if it is missing info
            return None #writing NaN
        else:
            return float(num); #write the float number
    else: num is None
    
#this function distinguish is there is a substring that we need in string
def parse_str_info( string, substring ):
    return 1 if re.findall(substring, string) else 0
    
#this function is used for loading data about one flat - flat which url is 
#http://www.cian.ru/sale/flat/%pagenumber%/ 
def go_to_page( page_number ):
    flat_url = 'http://www.cian.ru/sale/flat/' + str(links[i]) + '/'
    flat_page = requests.get(flat_url)
    flat_page = flat_page.content
    flat_page = BeautifulSoup(flat_page, 'lxml')  
    return flat_page;

#this function load table with general information from every flat page
def load_table():
    table = flat_page.find('table', attrs = {'class':'object_descr_props'})
    table = html_stripper(table)
    general_info = table.split('\n')
    return general_info;

def count_distance_to_center(lattitude, longitude):
    kremlin_lattitude = 55.752121
    kremlin_longitude = 37.617664
    kilometers_in_one_degree = 63.995
    dist = math.sqrt(math.pow((kremlin_lattitude - lattitude),2) + math.pow((kremlin_longitude - longitude),2))
    dist = dist * kilometers_in_one_degree
    return dist

And now begins parsing the whole CIAN Moscow

In [37]:
#We need to look on EVERY page
for i in range(0,len(links)):
    #Go to page i
    flat_page = go_to_page(str(links[i]))                         
                      
    #Now we need to find info about number of rooms in every flat (IF 9 THEN STUDIYA)
    for script in flat_page.findAll('script'):
        #If we found a script, inside which we have an info about rooms_count then we found what we need
        res = re.search(r'rooms_count: (.*),', str(script.string))
        if res:
            rooms = int(res.groups()[0])
            total_space = float(re.search(r'total_area: (.*),', str(script.string)).groups()[0].replace(',','.'))
    flats_info[i]['Rooms'] = rooms
    
    #Price
    price = flat_page.find('meta', attrs = {"itemprop":"price"})
    flats_info[i]['Price'] = parse_quantitative_info(str(price))
    
    #All information about metro
    metro = flat_page.find('span', attrs = {'class':'object_item_metro_comment'})
    if metro is None:
        metro_dist = None
        metro_way = None
    else: 
        metro_dist = int(re.findall(r'([0-9]+)', str(metro))[0])
        metro_way = 1 if re.findall(r'пешком', str(metro)) else 0
    flats_info[i]['Metrdist'] = metro_dist
    flats_info[i]['Walk'] = metro_way
    
    #Coordinates
    coords = flat_page.find('div', attrs={'class':'map_info_button_extend'}).contents[1]
    coords = re.split('&amp|center=|%2C', str(coords))
    coords_list = []
    for item in coords:
        if item[0].isdigit():
            coords_list.append(item)
    lattitude = float(coords_list[0])
    longitude = float(coords_list[1])
    distance_to_center = count_distance_to_center(lattitude, longitude)
    flats_info[i]['Dist'] = distance_to_center;
    
    
    #table with general information about flat
    general_info = load_table()
    #loading information we needed from general information
    #print(general_info)
    total_space = None
    live_space = None
    kitchen_space = None
    floor_number = None
    total_floors = None
    balcony = None
    new_house = None
    brick_house = None
    telephone = None
    
    
    for j in range(0,len(general_info)):
        if general_info[j] == 'Общая площадь:':
            total_space = parse_quantitative_info(general_info[j+2])
            
        if general_info[j] == 'Жилая площадь:':
            live_space = parse_quantitative_info(general_info[j+2])
            
        if general_info[j] == 'Площадь кухни:':
            kitchen_space = parse_quantitative_info(general_info[j+2])
            
        if general_info[j] == 'Этаж:':
            floors_info = re.findall(r'[–0-9,]+',general_info[j+2])
            floor_number = int(floors_info[0])
            if len(floors_info) > 1:
                total_floors = int(re.findall(r'[–0-9,]+',general_info[j+2])[1])
                
        if general_info[j] == 'Балкон:':
                #there could be info as quantitative (number of balcons) either string ("есть", "нет")
                balcony_str_ok = parse_str_info(general_info[j+1],'есть')
                balcony_str_notok = parse_str_info(general_info[j+1],'нет')
                if (balcony_str_ok or balcony_str_notok):
                        balcony = 1 if balcony_str_ok else 0
                else:
                    balcony_num = parse_quantitative_info(general_info[j+1])
                    balcony = balcony_num
                                        
        if general_info[j] == 'Тип дома:':
            new_house = parse_str_info(general_info[j+2],'новостройка')
            brick_house = parse_str_info(general_info[j+3],'кирпичный|монолитный|кирпично-монолитный|жб|железобетонный')
            
        if general_info[j] == 'Телефон:':
            telephone = parse_str_info(general_info[j+1], 'да') 
            
            
    flats_info[i]['Totsp'] = total_space        
    flats_info[i]['Livesp'] = live_space
    flats_info[i]['Kitsp'] = kitchen_space
    flats_info[i]['Floor'] = floor_number
    flats_info[i]['Nfloor'] = total_floors
    flats_info[i]['Bal'] = balcony
    flats_info[i]['New'] = new_house
    flats_info[i]['Brick'] = brick_house
    flats_info[i]['Tel'] = telephone
    if i % 100 == 0:
        print(i, 'flats loaded')

8400 flats loaded
8500 flats loaded
8600 flats loaded
8700 flats loaded
8800 flats loaded
8900 flats loaded
9000 flats loaded
9100 flats loaded
9200 flats loaded


In [38]:
print(flats_info[0:3])

[{'Brick': 0, 'New': 0, 'Nfloor': 10, 'Flat URL': '150919303', 'Dist': 1.7817103781050454, 'Floor': 7, 'Kitsp': 15.0, 'Totsp': 99.0, 'Metrdist': 10, 'Livesp': 80.0, 'N': 0, 'Rooms': 2, 'Bal': 1.0, 'Walk': 1, 'Price': 68000000.0, 'Tel': 0}, {'Brick': 1, 'New': 1, 'Nfloor': 8, 'Flat URL': '150425454', 'Dist': 1.7433886580310984, 'Floor': 6, 'Kitsp': 20.0, 'Totsp': 78.5, 'Metrdist': 12, 'Livesp': 50.0, 'N': 1, 'Rooms': 2, 'Bal': 1.0, 'Walk': 1, 'Price': 27786800.0, 'Tel': None}, {'Brick': 1, 'New': 1, 'Nfloor': 9, 'Flat URL': '149914299', 'Dist': 2.978800870726823, 'Floor': 6, 'Kitsp': 14.0, 'Totsp': 148.0, 'Metrdist': 7, 'Livesp': 57.0, 'N': 2, 'Rooms': 3, 'Bal': 2.0, 'Walk': 1, 'Price': 48000000.0, 'Tel': None}]


In [39]:
Moscow_flats = pd.DataFrame(flats_info)
Moscow_flats.to_csv('Moscow_flats.csv')